In [39]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [40]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [41]:
df_infos = pd.read_csv('../saved/infos.csv')
fDataReNorms = np.load('../saved/fDataReNorms.npy')

In [42]:
df_infos

,Unnamed: 0,ptID,Sex,Age,AF,I-AVB,LBBB,Normal,PAC,PVC,RBBB,STD,STE,mean_intervals,std_intervals,interval_len
0,0,A0001,0,74,False,False,False,False,False,False,True,False,False,299.055556,32.777825,303
1,1,A0001,0,74,False,False,False,False,False,False,True,False,False,299.055556,32.777825,292
2,2,A0001,0,74,False,False,False,False,False,False,True,False,False,299.055556,32.777825,296
3,3,A0001,0,74,False,False,False,False,False,False,True,False,False,299.055556,32.777825,296
4,4,A0001,0,74,False,False,False,False,False,False,True,False,False,299.055556,32.777825,298
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104423,104423,A6813,0,60,False,False,True,False,False,False,False,False,False,511.076923,20.409129,529
104424,104424,A6813,0,60,False,False,True,False,False,False,False,False,False,511.076923,20.409129,526
104425,104425,A6813,0,60,False,False,True,False,False,False,False,False,False,511.076923,20.409129,516
104426,104426,A6813,0,60,False,False,True,False,False,False,False,False,False,511.076923,20.409129,488


In [5]:
# from sklearn.preprocessing import normalize
#fDataReNorms=[fDataRe-np.tile(fDataRe[:,0],(250,1)).transpose() for fDataRe in fDataRes]
# fDataReNorms=np.array([normalize(fDataRe) for fDataRe in fDataRes])
# np.save('../saved/fDataReNorms.npy', fDataReNorms)
#[plt.plot(fDataReNorm[6]) for fDataReNorm in fDataReNorms]

In [6]:
# fDataReNormAFs = fDataReNorms[df_infos['AF']]

# for fDataReNormAF in fDataReNormAFs[:10]:
#     plt.plot(fDataReNormAF[0])
#     plt.show()

In [8]:
from collections import Counter
from global_vars import headers, labels
print(Counter(np.sum(df_infos[labels], axis=1)))

Counter({1: 96211, 2: 8079, 3: 138})


In [17]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
#torch.backends.cudnn.enabled = False

if False and torch.cuda.is_available():
    device = torch.device('cuda:0')
else:
    device = torch.device('cpu')

#torch.cuda.set_device(0)

In [33]:
y = torch.DoubleTensor(df_infos[labels].to_numpy()).to(device)

In [23]:
X = torch.DoubleTensor(fDataReNorms).to(device)

In [31]:
X2 = torch.DoubleTensor(df_infos[['Sex', 'Age', 'mean_intervals', 'std_intervals', 'interval_len']].to_numpy()).to(device)

In [12]:
class CNNWithProp(nn.Module):
    def __init__(self, verbose=False):
        super(CNNWithProp, self).__init__()
        self.conv1        = nn.Conv1d(12, 3, 20)
        self.max1         = nn.MaxPool1d(kernel_size=6)
#         self.conv2        = nn.Conv1d(3, 1, 10)
#         self.max2         = nn.MaxPool1d(kernel_size=6)
        self.fc3        = nn.Linear(119, 9)
        self.dropout = nn.Dropout(p=0.5)
        self.verbose=verbose

    def forward(self, x, x2):
        if self.verbose:
            print('0: ', x.shape)
        x = self.conv1( x )  
        x = F.relu(x)
        if self.verbose:
            print('1: ', x.shape)
        x = self.max1(x)
        if self.verbose:
            print('2: ', x.shape)
#         x = self.conv2( x )  
#         x = F.relu(x)
#         if self.verbose:
#             print('3: ', x.shape)
#         x = self.max2(x)
        if self.verbose:
            print('4: ', x.shape)
        x = x.view(x.shape[0], -1)
        x = torch.cat((x, x2),1)
        if self.verbose:
            print('5: ', x.shape)
        x = F.relu(x)
        if self.verbose:
            print('6: ', x.shape)
        x = self.fc3( x )  
        
        return x

# model = CNNWithProp(verbose=True).to(device).double()
# output = model.forward(X, X2)
# print(output)

In [37]:
from torch.utils.data import TensorDataset, DataLoader, WeightedRandomSampler
all_dataset = TensorDataset(X, X2, y)

In [38]:
class_weights = 1/np.sum(df_infos[labels],axis=0)
batch_size = 2000
sampler = WeightedRandomSampler(class_weights, batch_size)
all_loader = DataLoader(all_dataset, batch_size = batch_size, sampler = sampler)

https://discuss.pytorch.org/t/how-to-prevent-overfitting/1902

In [13]:
#%%
from sklearn.metrics import roc_auc_score, precision_recall_curve, auc
from evaluate_12ECG_score import compute_beta_score, compute_auc

# def evaluate_beta(output, y):
    
#     accuracy,f_measure,f_beta,g_beta = compute_beta_score(labels=y, 
#                        output=output, 
#                        beta=2, num_classes=1)
    
#     auroc, auprc = compute_auc(labels=y, 
#                                 probabilities=output,
#                                 num_classes=1)

#     return accuracy,f_measure,f_beta,g_beta, auroc, auprc


def confusion(prediction, truth):
    
    """ Returns the confusion matrix for the values in the `prediction` and `truth`
    tensors, i.e. the amount of positions where the values of `prediction`
    and `truth` are
    - 1 and 1 (True Positive)
    - 1 and 0 (False Positive)
    - 0 and 0 (True Negative)
    - 0 and 1 (False Negative)
    https://gist.github.com/the-bass/cae9f3976866776dea17a5049013258d
    """

    confusion_vector = prediction / truth
    # Element-wise division of the 2 tensors returns a new tensor which holds a
    # unique value for each case:
    #   1     where prediction and truth are 1 (True Positive)
    #   inf   where prediction is 1 and truth is 0 (False Positive)
    #   nan   where prediction and truth are 0 (True Negative)
    #   0     where prediction is 0 and truth is 1 (False Negative)

    true_positives = torch.sum(confusion_vector == 1).item()
    false_positives = torch.sum(confusion_vector == float('inf')).item()
    true_negatives = torch.sum(torch.isnan(confusion_vector)).item()
    false_negatives = torch.sum(confusion_vector == 0).item()

    return true_positives, false_positives, true_negatives, false_negatives

def binary_acc(y_preds, y_tests, beta=2):
    accs = []
    fmeasures = []
    fbetas = []
    gbetas = []
    aurocs = []
    auprcs = []
    for i in range(9):
        y_pred, y_test = y_preds[:,i], y_tests[:,i]
        
        # prob
        y_pred_prob = torch.sigmoid(y_pred)
        
        # auroc
        y_test_numpy = y_test.data.cpu().numpy()
        y_pred_prob_numpy = y_pred_prob.data.cpu().numpy()
        auroc = roc_auc_score(y_test_numpy, y_pred_prob_numpy)
        
        # auprc
        precision, recall, thresholds = precision_recall_curve(y_test_numpy, y_pred_prob_numpy)
        auprc = auc(recall, precision)

        # binary result
        y_pred_tag = torch.round(y_pred_prob)
        tp, fp, tn, fn = confusion(y_pred_tag, y_test)
        
        # acc, fmeasure, fbeta, gbeta
        acc = float(tp + tn) / float(tp + fp + fn + tn)
        fmeasure = float(2 * tp) / float(2 * tp + fp + fn)
        fbeta = float((1+beta**2)* tp) / float(((1+beta**2)*tp) + (fn*beta**2) + fp)
        gbeta = float(tp) / float(tp + fp + beta*fn)
        
        # old way to cal acc:
        #correct_results_sum = (y_pred_tag == y_test).sum().float()
        #acc = true_positives/y_test.shape[0]
        #acc = torch.round(acc * 100)

        accs.append(acc)#.data.cpu().numpy())
        fbetas.append(fbeta)#.data.cpu().numpy())
        fmeasures.append(fmeasure)
        gbetas.append(gbeta)
        aurocs.append(auroc)
        auprcs.append(auprc)
    return np.mean(accs), np.mean(fbetas), np.mean(fmeasures), np.mean(gbetas), np.mean(aurocs), np.mean(auprcs)

from snippets.pytorchtools import EarlyStopping
from sklearn.model_selection import KFold
import time

st = time.time()
patience = 2000
kf = KFold(10)

saved_dir = '../saved/model/'
for i, (train_idx, test_idx) in enumerate(kf.split(X, y)):
    model = CNNWithProp().to(device)
    model.double()
    learning_rate = 0.01
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    criterion = nn.BCEWithLogitsLoss()

    losses_train = []
    losses_test = []

    avg_losses_train = []
    avg_losses_test = []


    early_stopping = EarlyStopping(patience, verbose=False, 
                                  saved_dir=saved_dir, 
                                   save_name='CNNWithProp'+str(i))
    epoch = 0
    auroc = 0
    auprc = 0
    accuracy = 0
    fmeasure = 0
    fbeta = 0
    gbeta = 0
    for epoch in range(100000):
        
        train_loss = 0
        validation_loss = 0

        optimizer.zero_grad()
        output_train = model(X[train_idx], X2[train_idx])
        output_test = model(X[test_idx], X2[test_idx])

        loss_train = criterion(output_train, y[train_idx])
        loss_test = criterion(output_test, y[test_idx])

        loss_train.backward()
        optimizer.step()

        losses_train.append(loss_train.item())
        losses_test.append(loss_test.item())

        avg_loss_train = np.average(losses_train)
        avg_loss_test = np.average(losses_test)

        avg_losses_train.append(avg_loss_train)
        avg_losses_test.append(avg_loss_test)

        if epoch % 100 == 0:
            acc, fmeasure, fbeta, gbeta, auroc, auprc = binary_acc(output_train, y[train_idx])
            acc2, fmeasure2, fbeta2, gbeta2, auroc2, auprc2 = binary_acc(output_test, y[test_idx])
            #accuracy_p = ' '.join(['{:.2f},'.format(acc)for acc in accuracy])
            #accuracy2_p = ' '.join(['{:.2f},'.format(acc)for acc in accuracy2])
            print('S{}/{} {:.2f} min|\nTrain Loss: {:.6f}, Acc: {:.2f}, F: {:.2f}, Fbeta: {:.2f}, gbeta: {:.2f}, auroc: {:.2f}, auprc: {:.2f} |\nValid Loss: {:.6f}, Acc: {:.2f}, F: {:.2f}, Fbeta: {:.2f}, gbeta: {:.2f}, auroc: {:.2f}, auprc: {:.2f}\n '.format(
                i, epoch, (time.time()-st)/60,
                avg_loss_train, acc, fmeasure, fbeta, gbeta, auroc, auprc, 
                avg_loss_test, acc2, fmeasure2, fbeta2, gbeta2, auroc2, auprc2))

        early_stopping(avg_loss_test, model)

        if early_stopping.early_stop:
            print("Early stopping")
            break
            
    output_string = 'AUROC|AUPRC|Accuracy|F-measure|Fbeta-measure|Gbeta-measure\n{:.3f}|{:.3f}|{:.3f}|{:.3f}|{:.3f}|{:.3f}'.format(auroc,auprc,acc,fmeasure,fbeta,gbeta)
    print(output_string)     
    with open(saved_dir+'score'+ str(i)+ '_epoch' + str(epoch) + '.txt', 'w') as f:
        f.write(output_string)

    avg_losses_train = np.array(avg_losses_train)
    avg_losses_test = np.array(avg_losses_test)
    
    np.save(saved_dir + 'avg_losses_train' + str(i) + '_epoch' + str(epoch), avg_losses_train)
    np.save(saved_dir + 'avg_losses_test' + str(i) + '_epoch' + str(epoch), avg_losses_test)
    

RuntimeError: cuDNN error: CUDNN_STATUS_NOT_INITIALIZED

In [ ]:
torch.backends.cudnn.version()

In [ ]:
import torch.__config__
print(torch.__config__.show())